In [18]:
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from pandas import DataFrame
import pandas as pd
import seaborn as sns
import numpy as np
import json
import os
import tensorflow as tf

np.set_printoptions(threshold=np.inf)

fileName = 'DSL-StrongPasswordData.xls'
df_oral = pd.read_excel(fileName)
df_oral = df_oral[(df_oral['subject']== 's002') | (df_oral['subject']== 's032') | (df_oral['subject']== 's036') | (df_oral['subject']== 's047') | (df_oral['subject']== 's052')] 

ud_features = [column for column in df_oral.columns if 'UD' in column]
dd_features = [column for column in df_oral.columns if 'DD' in column]

#object_user = 's002'
#object_user = 's032'
object_user = 's036'
#object_user = 's047'
#object_user = 's052'

df_oral.loc[df_oral['subject']!=object_user,'subject'] = 'imposter'
df = df_oral

#df = df_oral.loc[(df_oral['subject']==object_user) | (df_oral['sessionIndex']<=4)]

#print (df) 
###################################PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,LabelEncoder,LabelEncoder

le = LabelEncoder()
df['target'] = le.fit_transform(df['subject'])
df.drop(['subject'], axis=1, inplace=True)

X = df.drop(['target','rep','sessionIndex'], axis=1)[ud_features+dd_features]
y = df['target'].values

ss = StandardScaler()
X = ss.fit_transform(X)
X = pd.DataFrame(X)

将样本分为训练集和测试集

In [20]:
'''
#s002
index_train1 = [i for i in range(10)]    #s002
index_train2 = [i+400 for i in range(3)] #s032
index_train3 = [i+800 for i in range(3)] #s036
index_train4 = [i+1200 for i in range(3)] #s047
index_train5 = [i+1600 for i in range(3)] #s052

index_test1 = [i+200 for i in range(200)] #s002
index_test2 = [i+450 for i in range(350)]  #s032
index_test3 = [i+850 for i in range(350)]  #s036
index_test4 = [i+1250 for i in range(350)]  #s047
index_test5 = [i+1650 for i in range(350)]  #s052
'''

'''
#s032
index_train1 = [i+400 for i in range(10)] #s032
index_train2 = [i for i in range(3)] #s002
index_train3 = [i+800 for i in range(3)] #s036
index_train4 = [i+1200 for i in range(3)] #s047
index_train5 = [i+1600 for i in range(3)] #s052

index_test1 = [i+600 for i in range(200)] #s032
index_test2 = [i+50 for i in range(350)]  #s002
index_test3 = [i+850 for i in range(350)]  #s036
index_test4 = [i+1250 for i in range(350)]  #s047
index_test5 = [i+1650 for i in range(350)]  #s052
'''


#s036
index_train1 = [i+800 for i in range(10)] #s036
index_train2 = [i for i in range(3)] #s002
index_train3 = [i+400 for i in range(3)] #s032
index_train4 = [i+1200 for i in range(3)] #s047
index_train5 = [i+1600 for i in range(3)] #s052

index_test1 = [i+1000 for i in range(200)] #s036
index_test2 = [i+50 for i in range(350)]  #s002
index_test3 = [i+450 for i in range(350)]  #s032
index_test4 = [i+1250 for i in range(350)]  #s047
index_test5 = [i+1650 for i in range(350)]  #s052


'''
#s047
index_train1 = [i+1200 for i in range(10)] #s047
index_train2 = [i for i in range(3)] #s002
index_train3 = [i+400 for i in range(3)] #s032
index_train4 = [i+800 for i in range(3)] #s036
index_train5 = [i+1600 for i in range(3)] #s052

index_test1 = [i+1400 for i in range(200)] #s047
index_test2 = [i+50 for i in range(350)]  #s002
index_test3 = [i+450 for i in range(350)]  #s032
index_test4 = [i+850 for i in range(350)]  #s036
index_test5 = [i+1650 for i in range(350)]  #s052
'''

'''
#s052
index_train1 = [i+1600 for i in range(10)] #s052
index_train2 = [i for i in range(3)] #s002
index_train3 = [i+400 for i in range(3)] #s032
index_train4 = [i+900 for i in range(3)] #s036
index_train5 = [i+1200 for i in range(3)] #s047

index_test1 = [i+1800 for i in range(200)] #s052
index_test2 = [i+50 for i in range(350)]  #s002
index_test3 = [i+450 for i in range(350)]  #s032
index_test4 = [i+850 for i in range(350)]  #s036
index_test5 = [i+1250 for i in range(350)]  #s047
'''

index_train = index_train1 + index_train2 + index_train3 + index_train4 + index_train5
index_test = index_test1 + index_test2 + index_test3 + index_test4 + index_test5

X_train = X.iloc[index_train]
X_test = X.iloc[index_test]
y_train = y[index_train]
y_test = y[index_test]

SVM2

In [21]:
from numpy.random import seed
seed(1)
tf.random.set_seed(2)

import numpy as np
import sklearn.model_selection as ms
import sklearn.svm as svm
import sklearn.metrics as sm
import matplotlib.pyplot as plt

# 最优超参数组合列表
params = [
        {'kernel': ['linear'], 'C': [1, 10, 100, 100,1000]},
        {'kernel': ['poly'], 'C': [1,10,100], 'degree': [2, 3]},
        {'kernel': ['rbf'], 'C': [1, 10, 100, 100,1000], 'gamma':[1, 0.1, 0.01, 0.001]}
        ]

model = ms.GridSearchCV(svm.SVC(probability=True), 
                        params, 
                        refit=True,
                        return_train_score=True,        # 后续版本需要指定True才有score方法  
                        cv=4)
model.fit(X_train, y_train)

# GridSearchCV的属性
print('Attrabutes:vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv')
print('cv_results_:',model.cv_results_.keys())
print('Desc:',model.cv_results_['params'][2], model.cv_results_['mean_train_score'][2], 
      model.cv_results_['mean_test_score'][2],model.cv_results_['rank_test_score'][2])
print('best_estimator_:',model.best_estimator_) 
print('best_params_:',model.best_params_) 
print('best_params_:', model.cv_results_['params'][model.best_index_])
print('best_score_:',model.best_score_)
print('scorer_:',model.scorer_)
print('n_splits_:',model.n_splits_)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:196: FutureWarning: The default 

Attrabutes:vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
cv_results_: dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_C', 'param_kernel', 'param_degree', 'param_gamma', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score', 'split3_train_score', 'mean_train_score', 'std_train_score'])
Desc: {'C': 100, 'kernel': 'linear'} 1.0 0.9090909090909091 2
best_estimator_: SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
best_params_: {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
best_params_: {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
best_score_: 0.9545454545454546
scorer_: <function _passthrough_scorer at 0x7effff92a1e0>
n_splits_: 

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [22]:
y_predict_SVM = model.best_estimator_.predict(X_test)

ann_fileName = "SVM_best"+object_user+"_CMU_pred_final_10.txt"
np.savetxt(ann_fileName, np.array(y_predict_SVM))

#评价指标
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score,accuracy_score

y_true = np.reshape(y_test, [-1])
y_pred = np.reshape(y_predict_SVM, [-1])
  
# 评价指标
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='binary')
f1score = f1_score(y_true, y_pred, average='binary')
  
# Micro F1: 将n分类的评价拆成n个二分类的评价，将n个二分类评价的TP、FP、RN对应相加，计算评价准确率和召回率，由这2个准确率和召回率计算的F1 score即为Micro F1。
# Macro F1: 将n分类的评价拆成n个二分类的评价，计算每个二分类的F1 score，n个F1 score的平均值即为Macro F1。
# 一般来讲，Macro F1、Micro F1高的分类效果好。Macro F1受样本数量少的类别影响大。
micro_f1 = f1_score(y_true, y_pred,average='micro')
macro_f1 = f1_score(y_true, y_pred,average='macro')
  
print('accuracy:',accuracy)
print('precision:',precision)
print('recall:',recall)
print('f1score:',f1score)
print('Macro-F1: {}'.format(macro_f1))
print('Micro-F1: {}'.format(micro_f1))

accuracy: 0.883125
precision: 0.9333333333333333
recall: 0.07
f1score: 0.13023255813953488
Macro-F1: 0.5337929959876904
Micro-F1: 0.8831250000000002


神经网络

In [23]:
from numpy.random import seed
seed(1)
tf.random.set_seed(2)

from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np
import pickle
import gzip
import matplotlib.pyplot as plt

mlp_clf__tuned_parameters = {"hidden_layer_sizes": [(40,),(40,32),(64,),(64,32)],
                             "solver": ['adam', 'sgd', 'lbfgs'],
                             "max_iter": [20,100,200,1000,5000],
                             "verbose": [True]
                             }
mlp = MLPClassifier()
estimator = GridSearchCV(mlp, mlp_clf__tuned_parameters, n_jobs=6)
estimator.fit(X_train, y_train)

print(estimator.get_params().keys()) 
print (estimator.best_params_)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Iteration 1, loss = 0.87089569
Iteration 2, loss = 0.84985588
Iteration 3, loss = 0.82239534
Iteration 4, loss = 0.79160474
Iteration 5, loss = 0.75993309
Iteration 6, loss = 0.72954707
Iteration 7, loss = 0.70161981
Iteration 8, loss = 0.67689148
Iteration 9, loss = 0.65517257
Iteration 10, loss = 0.63624277
Iteration 11, loss = 0.61963721
Iteration 12, loss = 0.60491818
Iteration 13, loss = 0.59152409
Iteration 14, loss = 0.57893612
Iteration 15, loss = 0.56683956
Iteration 16, loss = 0.55495596
Iteration 17, loss = 0.54311600
Iteration 18, loss = 0.53123923
Iteration 19, loss = 0.51937185
Iteration 20, loss = 0.50753858
Iteration 21, loss = 0.49581305
Iteration 22, loss = 0.48429681
Iteration 23, loss = 0.47309310
Iteration 24, loss = 0.46237016
Iteration 25, loss = 0.45210283
Iteration 26, loss = 0.44234939
Iteration 27, loss = 0.43313960
Iteration 28, loss = 0.42447044
Iteration 29, loss = 0.41628612
Iteration 30, loss = 0.40858348
Iteration 31, loss = 0.40138379
Iteration 32, los

Iteration 318, loss = 0.04377107
Iteration 319, loss = 0.04357308
Iteration 320, loss = 0.04337629
Iteration 321, loss = 0.04318101
Iteration 322, loss = 0.04298693
Iteration 323, loss = 0.04279430
Iteration 324, loss = 0.04260289
Iteration 325, loss = 0.04241328
Iteration 326, loss = 0.04222472
Iteration 327, loss = 0.04203699
Iteration 328, loss = 0.04185057
Iteration 329, loss = 0.04166566
Iteration 330, loss = 0.04148213
Iteration 331, loss = 0.04129961
Iteration 332, loss = 0.04112073
Iteration 333, loss = 0.04094340
Iteration 334, loss = 0.04076711
Iteration 335, loss = 0.04059198
Iteration 336, loss = 0.04041821
Iteration 337, loss = 0.04024532
Iteration 338, loss = 0.04007386
Iteration 339, loss = 0.03990355
Iteration 340, loss = 0.03973408
Iteration 341, loss = 0.03956539
Iteration 342, loss = 0.03939759
Iteration 343, loss = 0.03923103
Iteration 344, loss = 0.03906550
Iteration 345, loss = 0.03890100
Iteration 346, loss = 0.03873760
Iteration 347, loss = 0.03857553
Iteration 

In [13]:
from numpy.random import seed
seed(1)
tf.random.set_seed(2)

# two-layer neural network 
# train part
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold
from joblib import dump

#neural network classifier of structure (3,2)
kf = KFold(n_splits=4) # 3-fold cross-validation
best_clf = None
best_score = 0
train_scores = []
test_scores = []
print("kfold-------")
for train_index, test_index in kf.split(X):
    # create neural network using MLPClassifer  {sgd,lbfgs, adam}
    clf = MLPClassifier(solver = 'sgd', max_iter =1000, hidden_layer_sizes =(40, 32),random_state = 1)
    #X_train, X_test = X[train_index], X[test_index]
    #y_train, y_test = y[train_index], y[test_index]
    clf.fit(X_train, y_train)
    train_score = clf.score(X_train, y_train)
    train_scores.append(train_score)
 
    test_score = clf.score(X_test, y_test)
    test_scores.append(test_score)

    #compare score of the tree models and get the best one
    if test_score > best_score:
        best_score = test_score
        best_clf = clf
    
    #print(clf.n_outputs_)
in_sample_error = [1 - score for score in train_scores]
test_set_error = [1 - score for score in test_scores]
print("in_sample_error: ")
print(in_sample_error)
print("test_set_error: ")
print(test_set_error)

#store the classifier
if best_clf != None:
    dump(best_clf, "train_model.m")
    
from joblib import load
clf = load("train_model.m")
y_pred = clf.predict(X_test)
np.savetxt("label_pred.txt", np.array(y_pred)) #save predict result
#print(y_pred)
test_score = clf.score(X_test, y_test)
test_error = 1 - test_score
print('test_score：%s' % test_score)
print('test_error：%s' % test_error)

kfold-------
in_sample_error: 
[0.0, 0.0, 0.0, 0.0]
test_set_error: 
[0.24624999999999997, 0.24624999999999997, 0.24624999999999997, 0.24624999999999997]
test_score：0.75375
test_error：0.24624999999999997


In [16]:
#评价指标
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score,accuracy_score

y_true = np.reshape(y_test, [-1])
y_pred = np.reshape(y_pred, [-1])
  
# 评价指标
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='binary')
f1score = f1_score(y_true, y_pred, average='binary')
  
# Micro F1: 将n分类的评价拆成n个二分类的评价，将n个二分类评价的TP、FP、RN对应相加，计算评价准确率和召回率，由这2个准确率和召回率计算的F1 score即为Micro F1。
# Macro F1: 将n分类的评价拆成n个二分类的评价，计算每个二分类的F1 score，n个F1 score的平均值即为Macro F1。
# 一般来讲，Macro F1、Micro F1高的分类效果好。Macro F1受样本数量少的类别影响大。
micro_f1 = f1_score(y_true, y_pred,average='micro')
macro_f1 = f1_score(y_true, y_pred,average='macro')
  
print('accuracy:',accuracy)
print('precision:',precision)
print('recall:',recall)
print('f1score:',f1score)
print('Macro-F1: {}'.format(macro_f1))
print('Micro-F1: {}'.format(micro_f1))

accuracy: 0.75375
precision: 0.32922535211267606
recall: 0.935
f1score: 0.48697916666666674
Macro-F1: 0.6624862938596492
Micro-F1: 0.75375


In [17]:
ann_fileName = "ANN_"+object_user+"_CMU_pred_final_10.txt"
np.savetxt(ann_fileName, np.array(y_pred))